## What was discovered?
* is_tourney (remark when game was in regular or tourney phase) variable doesnt matter according to feature importance and also we will predict only tourney phase games.
* to addres the problem of data leakage I removed scores from games (before game we dont know which teams shoot how many points) and also for every team i retrieve seed values. But if it is regular season i only have information about previous ones. So for regular season I retrieve seed from previous season and for tourney phase i retrieve information from current season.
* team coaches can have also some important informations (some coaches are better than other). But during the season team coache can be changed. So apart classic merge I have to check also if in particular row daynum is smaller than last day and bigger than first day of coach term.
* DayNum feature can carry some information. But in this contest we do not have information about matches (we have to predict every possible matchup). Feature importance indicate it is important feature.
* Ranking provide a lot of information. In data we have a massive ranking, which contains a team's ranking from a lot of rankings. There is above 100 rankings. So i will caluclate median for certain game for particular team. 

## Which features was added?
* seed value for every team
* difference between seed of one team and another
* team coach information
* duration of teams presence in Division I
* ranking's median

## Load libraries

In [1]:
import fireducks.pandas as pd
import numpy as np
from sklearn.metrics import brier_score_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from custom_utils import transform_data, enrich_data, feature_importance
from IPython.display import display
from sklearn.decomposition import PCA
from collections import defaultdict

## Load and prepare data

In [14]:
wregularseason = pd.read_csv("data/WRegularSeasonCompactResults.csv")
wtourneyseason = pd.read_csv("data/WNCAATourneyCompactResults.csv")

mregularseason = pd.read_csv("data/MRegularSeasonCompactResults.csv")
mtourneyseason = pd.read_csv("data/MNCAATourneyCompactResults.csv")

mseason = pd.read_csv("data/MSeasons.csv")
wseason = pd.read_csv("data/WSeasons.csv")

mrankings = pd.read_csv("data/MMasseyOrdinals.csv")
mrankings = mrankings[mrankings.Season > 2015].reset_index(drop=True)

wprep = pd.concat([transform_data(wregularseason), transform_data(wtourneyseason)], axis=0).reset_index(drop=True)
mprep = pd.concat([transform_data(mregularseason), transform_data(mtourneyseason)], axis=0).reset_index(drop=True)

In [15]:
medianrankings = mrankings.groupby(["Season", "RankingDayNum", "TeamID"]).agg({"OrdinalRank": "median"}).reset_index()
medianrankings

,Season,RankingDayNum,TeamID,OrdinalRank
0,2016,16,1101,342.0
1,2016,16,1102,202.0
2,2016,16,1103,77.0
3,2016,16,1104,96.0
4,2016,16,1105,307.0
...,...,...,...,...
64474,2025,133,1476,316.5
64475,2025,133,1477,340.5
64476,2025,133,1478,349.5
64477,2025,133,1479,329.0


In [17]:
mprep_enh = enrich_data(mprep, "M")
wprep_enh = enrich_data(wprep, "W")
wprep_enh = wprep_enh.loc[wprep_enh["Season"] > 2015].reset_index(drop=True)
mprep_enh = mprep_enh.loc[mprep_enh["Season"] > 2015].reset_index(drop=True)
display(wprep_enh.head())
mprep_enh.head()

/home/kacper/Projects/Forecasting-NCAA-BB/venv/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kacper/Projects/Forecasting-NCAA-BB/venv/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


,Season,DayNum,WTeamID,LTeamID,Result,is_tournament,SeedW,SeedL,SeedDiff
0,2025,122,3303,3355,0,0,16,12,4
1,2025,122,3264,3357,0,0,16,16,0
2,2025,122,3413,3414,0,0,16,13,3
3,2025,122,3324,3453,0,0,16,11,5
4,2025,123,3202,3151,0,0,16,14,2


,Season,DayNum,WTeamID,LTeamID,Result,is_tournament,SeedW,SeedL,WCoachName,LCoachName,Duration,SeedDiff
0,2023,72,1395,1400,0,0,9,6,632,1325,39.0,3
1,2023,73,1276,1234,0,0,11,5,860,497,39.0,6
2,2023,74,1277,1228,0,0,7,4,1528,162,39.0,3
3,2023,75,1261,1104,0,0,6,6,1039,1141,39.0,0
4,2023,75,1461,1129,0,0,12,8,669,962,39.0,4


In [19]:
mprep_enh_rank = mprep_enh.merge(
    medianrankings,
    how="left",
    left_on=["Season", "WTeamID"], 
    right_on=["Season", "TeamID"]
)

mprep_enh_rank = mprep_enh_rank[mprep_enh_rank.RankingDayNum - mprep_enh_rank.DayNum <= 0].reset_index(drop=True)
mprep_enh_rank = mprep_enh_rank.groupby(["Season", "DayNum", "WTeamID", "LTeamID"]).agg(lambda x: x.iloc[-1]).reset_index()\
    .drop(["RankingDayNum", "TeamID"], axis=1).rename(columns={"OrdinalRank": "RankW"})

mprep_enh_rank = mprep_enh_rank.merge(
    medianrankings,
    how="left",
    left_on=["Season", "LTeamID"], 
    right_on=["Season", "TeamID"]
)

mprep_enh_rank = mprep_enh_rank[mprep_enh_rank.RankingDayNum - mprep_enh_rank.DayNum <= 0].reset_index(drop=True)
mprep_enh_rank = mprep_enh_rank.groupby(["Season", "DayNum", "WTeamID", "LTeamID"]).agg(lambda x: x.iloc[-1]).reset_index()\
    .drop(["RankingDayNum", "TeamID"], axis=1).rename(columns={"OrdinalRank": "RankL"})
mprep_enh_rank.head()

,Season,DayNum,WTeamID,LTeamID,Result,is_tournament,SeedW,SeedL,WCoachName,LCoachName,Duration,SeedDiff,RankW,RankL
0,2016,16,1103,1116,1,0,16,5,867,1070,32.0,11,77.0,45.0
1,2016,16,1113,1244,1,0,16,16,148,13,32.0,0,86.0,334.0
2,2016,16,1116,1103,0,0,5,16,1070,867,32.0,-11,45.0,77.0
3,2016,16,1117,1366,0,0,16,16,797,597,32.0,0,268.0,334.0
4,2016,16,1119,1192,1,0,16,16,1619,567,32.0,0,193.0,324.0


## Test models on different features

### Divide dataset on test and train sets

In [20]:
N_COMPONENTS = 3
pca = PCA(n_components=N_COMPONENTS)

feature_labels = [f"feature_{i}" for i in range(1, N_COMPONENTS + 1)]

X_featm = mprep_enh_rank.drop(["Result", "DayNum"], axis=1)
ym = mprep_enh_rank.Result

X_featw = wprep_enh.drop(["Result", "DayNum"], axis=1)
yw = wprep_enh.Result

X_trainm, X_testm, y_trainm, y_testm = train_test_split(X_featm, ym, test_size=0.1)
X_trainw, X_testw, y_trainw, y_testw = train_test_split(X_featw, yw, test_size=0.1)

X_trainm_pca = pd.DataFrame(pca.fit_transform(X_trainm), columns=feature_labels)
X_testm_pca = pd.DataFrame(pca.transform(X_testm), columns=feature_labels)

X_trainw_pca = pd.DataFrame(pca.fit_transform(X_trainw), columns=feature_labels)
X_testw_pca = pd.DataFrame(pca.transform(X_testw), columns=feature_labels)

### Without PCA

In [21]:
rf_clfm = RandomForestClassifier()
rf_clfm.fit(X_trainm, y_trainm)

rf_clfw = RandomForestClassifier()
_ = rf_clfw.fit(X_trainw, y_trainw)

In [22]:
pred_probsm = rf_clfm.predict_proba(X_testm)[:, 1]
pred_probsw = rf_clfw.predict_proba(X_testw)[:, 1]

print(f"Brier score for mens data: {np.round(brier_score_loss(y_testm, pred_probsm), 3).item()}")
print(f"Brier score for womens data: {np.round(brier_score_loss(y_testw, pred_probsw), 3).item()}")

Brier score for mens data: 0.235
Brier score for womens data: 0.227


In [23]:
print("For men's games")
display(feature_importance(rf_clfm, mprep_enh_rank.drop("DayNum", axis=1)))

print("For women's games")
display(feature_importance(rf_clfw, wprep_enh.drop("DayNum", axis=1)))

For men's games
[0.00197089 0.0293812  0.02597365 0.00311523 0.02024485 0.01880903
 0.03586654 0.03241639 0.00236958 0.022381   0.03201896 0.02966541]


,Feature,Importance_clf,Importance_mi
0,Season,0.045773,0.001971
1,WTeamID,0.098763,0.029381
2,LTeamID,0.111893,0.025974
3,is_tournament,0.001275,0.003115
4,SeedW,0.016830,0.020245
5,SeedL,0.018192,0.018809
6,WCoachName,0.100398,0.035867
7,LCoachName,0.112800,0.032416
8,Duration,0.057883,0.002370
9,SeedDiff,0.035053,0.022381


For women's games
[0.0026809  0.03954748 0.04427954 0.00037893 0.02848336 0.02459424
 0.0445614 ]


,Feature,Importance_clf,Importance_mi
0,Season,0.064621,0.002681
1,WTeamID,0.410519,0.039547
2,LTeamID,0.406935,0.044280
3,is_tournament,0.001372,0.000379
4,SeedW,0.031427,0.028483
5,SeedL,0.028456,0.024594
6,SeedDiff,0.056669,0.044561


### With PCA

In [26]:
rf_clfm = RandomForestClassifier()
rf_clfm.fit(X_trainm_pca, y_trainm)

rf_clfw = RandomForestClassifier()
_ = rf_clfw.fit(X_trainw_pca, y_trainw)


pred_probsm = rf_clfm.predict_proba(X_testm_pca)[:, 1]
pred_probsw = rf_clfw.predict_proba(X_testw_pca)[:, 1]

print(f"Brier score for mens data: {np.round(brier_score_loss(y_testm, pred_probsm), 3).item()}")
print(f"Brier score for womens data: {np.round(brier_score_loss(y_testw, pred_probsw), 3).item()}")

Brier score for mens data: 0.271
Brier score for womens data: 0.237


In [27]:
wdatatrain = pd.concat([X_trainw_pca, y_trainw.reset_index(drop=True)], axis=1)
mdatatrain = pd.concat([X_trainm_pca, y_trainm.reset_index(drop=True)], axis=1)

wdatatest = pd.concat([X_testw_pca, y_testw.reset_index(drop=True)], axis=1)
mdatatest = pd.concat([X_testm_pca, y_testm.reset_index(drop=True)], axis=1)

wdata = pd.concat([wdatatrain, wdatatest], axis=0).reset_index(drop=True)
mdata = pd.concat([mdatatrain, mdatatest], axis=0).reset_index(drop=True)

print("For men's games")
display(feature_importance(rf_clfm, mdata))

print("For women's games")
display(feature_importance(rf_clfw, wdata))

For men's games
[0.02818481 0.03553904 0.02435911]


,Feature,Importance_clf,Importance_mi
0,feature_1,0.338186,0.028185
1,feature_2,0.335251,0.035539
2,feature_3,0.326563,0.024359


For women's games
[0.11460115 0.11527089 0.08057058]


,Feature,Importance_clf,Importance_mi
0,feature_1,0.311891,0.114601
1,feature_2,0.299254,0.115271
2,feature_3,0.388854,0.080571
